In [4]:
import pandas as pd

In [5]:
df=pd.read_csv("input.csv")

#convert columns at 1,2,3 to datetime
date_time_cols=[0,1,2,3]
for i in date_time_cols:
    df.iloc[:,i]=pd.to_datetime(df.iloc[:,i])

df=df.drop(df.columns[-1],axis=1)
df['time_diff']=(df['delivered_to_consumer_datetime']-df['customer_placed_order_datetime']).dt.total_seconds()/60



,customer_placed_order_datetime,placed_order_with_restaurant_datetime,driver_at_restaurant_datetime,delivered_to_consumer_datetime,driver_id,restaurant_id,consumer_id,is_new,delivery_region,is_asap,order_total,discount_amount,tip_amount,refunded_amount
0,2020-05-25 16:36:00,2020-05-25 18:17:00,NaT,2020-05-25 18:47:00,257,154,190327,True,San Jose,False,36.99,6,1.42,0.0
1,2020-05-02 16:32:00,2020-05-02 17:59:00,NaT,2020-05-02 18:50:00,168,297,12420,False,San Jose,False,32.71,0,3.67,0.0
2,2020-05-24 16:38:00,2020-05-24 20:03:00,2020-05-24 20:37:00,2020-05-24 20:49:00,290,201,106412,True,San Jose,False,33.35,6,1.25,0.0
3,2020-05-17 17:34:00,2020-05-17 18:01:00,2020-05-17 18:09:00,2020-05-17 18:35:00,232,188,104751,False,San Jose,False,33.02,6,2.48,0.0
4,2020-05-24 17:40:00,2020-05-24 18:04:00,NaT,2020-05-24 18:51:00,290,135,13641,True,San Jose,False,24.49,6,2.45,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2020-05-21 16:29:00,2020-05-21 17:36:00,2020-05-21 18:26:00,2020-05-21 18:37:00,320,8,110738,True,Palo Alto,False,38.57,6,1.49,0.0
92,2020-05-04 00:11:00,2020-05-04 00:28:00,2020-05-04 00:52:00,2020-05-04 01:21:00,313,28,8336,False,Palo Alto,False,48.74,0,2.44,0.0
93,2020-05-27 01:28:00,2020-05-27 01:44:00,NaT,2020-05-27 02:22:00,299,48,193268,False,Palo Alto,True,47.16,6,1.89,0.0
94,2020-05-23 00:27:00,2020-05-23 00:27:00,2020-05-23 00:35:00,2020-05-23 00:53:00,222,20,2978,False,Palo Alto,True,42.87,0,2.14,0.0


In [ ]:
#drop the last coulmn

res={}
for i in range(len(df)):
    row=df.iloc[i]
    if row.restaurant_id not in res:
        res[row.restaurant_id]=0
    res[row.restaurant_id]+=row.order_total
keys=list(res.keys())
values=list(res.values())
res=pd.DataFrame({"restaurant_id":keys,"total_revenue":values})
res.sort_values(by="total_revenue",ascending=True,inplace=True)
res.iloc[0:2].sort_values(by="restaurant_id",ascending=True)


In [12]:
result={}
for i in range(len(df)):
    row=df.iloc[i]
    if row.restaurant_id not in result:
        result[row.restaurant_id]={'order_total':[],'time_diff':[]}
    result[row.restaurant_id]['order_total'].append(row.order_total)
    result[row.restaurant_id]['time_diff'].append(row.time_diff)
res_id=list(result.keys())
totals=[]
time_diffs=[]
for i in res_id:
    totals.append(sum(result[i]['order_total'])/len(result[i]['order_total']))
    time_diffs.append(sum(result[i]['time_diff'])/len(result[i]['time_diff']))
res2=pd.DataFrame({"restaurant_id":res_id,"avg_order_total":totals,"avg_time_diff":time_diffs})
res2['avg_time_diff'].corr(res2['avg_order_total'])



0.859251984729398

,driver_id,restaurant_id,consumer_id,is_new,is_asap,order_total,discount_amount,tip_amount,refunded_amount,time_diff
driver_id,1.000000,-0.011058,0.057641,-0.142948,0.031558,-0.200858,0.080402,-0.011902,-0.095853,-0.166381
restaurant_id,-0.011058,1.000000,0.219944,-0.005658,-0.099565,-0.061489,0.037262,0.019608,-0.105409,-0.036495
consumer_id,0.057641,0.219944,1.000000,0.062513,-0.008133,0.015301,0.651861,-0.186469,-0.048024,0.070207
is_new,-0.142948,-0.005658,0.062513,1.000000,-0.137396,0.081254,0.050074,-0.228742,0.182783,0.151805
is_asap,0.031558,-0.099565,-0.008133,-0.137396,1.000000,-0.278353,-0.109364,-0.200279,0.064179,-0.461156
order_total,-0.200858,-0.061489,0.015301,0.081254,-0.278353,1.000000,-0.107037,0.220862,-0.012747,0.747862
discount_amount,0.080402,0.037262,0.651861,0.050074,-0.109364,-0.107037,1.000000,-0.130643,-0.055534,0.120673
tip_amount,-0.011902,0.019608,-0.186469,-0.228742,-0.200279,0.220862,-0.130643,1.000000,-0.047664,0.043993
refunded_amount,-0.095853,-0.105409,-0.048024,0.182783,0.064179,-0.012747,-0.055534,-0.047664,1.000000,-0.002363
time_diff,-0.166381,-0.036495,0.070207,0.151805,-0.461156,0.747862,0.120673,0.043993,-0.002363,1.000000
